In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Approaches Comparison

In [2]:
ea_df = pd.read_csv('data/ea/20,50,5,0.1,0.2_0.csv', index_col=0)
sa_df = pd.read_csv('data/sa/20.0,1.0,0.9,100,0.1,0.3_0.csv', index_col=0)

In [19]:
(ea_df.groupby(by='run').agg(
    total_time = ('time_elapsed', 'sum')
) / 1_000_000_000).reset_index()

,run,total_time
0,0.0,66.971538
1,1.0,62.632942
2,2.0,30.701099
3,3.0,10.633552
4,4.0,46.702646
5,5.0,49.310826
6,6.0,11.253086
7,7.0,5.995925
8,8.0,70.308873
9,9.0,8.426953


In [178]:
def get_fig(df, x_column, xlabel, ylabel="Reward", fig=None, color="#0000ff", name=""):
    mean_df = df.groupby(by=[x_column]).agg(
        mean_best_fitness = ('best_fitness', 'mean'),
        std_best_fitness = ('best_fitness', 'std'),
    ).fillna(0).reset_index()
    if fig is None:
        fig = go.Figure()
    # fig.add_trace(go.Scatter(
    #     x=mean_df[x_column],
    #     y=mean_df['mean_best_fitness'] + mean_df['std_best_fitness'],
    #     mode='lines',
    #     line=dict(color='#cccccc'),
    #     showlegend=False
    # ))
    # fig.add_trace(go.Scatter(
    #     x=mean_df[x_column],
    #     y=np.maximum(mean_df['mean_best_fitness'] - mean_df['std_best_fitness'], 0),
    #     mode='lines',
    #     line=dict(color='#cccccc'),
    #     fill='tonexty',
    #     opacity=0.01,
    #     showlegend=False
    # ))
    fig.add_trace(go.Scatter(
        x=mean_df[x_column],
        y=mean_df['mean_best_fitness'],
        mode='lines',
        line=dict(color=color),
        showlegend= name != "",
        name=name
    ))
    # for run in range(10):
    #     run_df = df.loc[df['run'] == run]
    #     fig.add_trace(go.Scatter(
    #         x=run_df[x_column],
    #         y=run_df['best_fitness'],
    #         mode='lines',
    #         line=dict(color=color),
    #         opacity=0.1,
    #         showlegend=False
    #     ))  
    fig.update_layout(go.Layout(
        width=800,
        margin=dict(l=20, r=20, t=20, b=20),
        xaxis_title=xlabel,
        yaxis_title=ylabel,
        xaxis_gridcolor='lightgrey',
        yaxis_gridcolor='lightgrey',
        plot_bgcolor='white',
    ))
    return fig

In [99]:
fig = get_fig(ea_df, x_column="generation", xlabel="Generations")
fig.show()

In [38]:
(sa_df.groupby(by='run').agg(
    total_time = ('time_elapsed', 'sum')
) / 1_000_000_000).reset_index()

,run,total_time
0,0.0,95.755459
1,1.0,22.236433
2,2.0,17.321755
3,3.0,17.469985
4,4.0,16.883925
5,5.0,20.436588
6,6.0,18.335797
7,7.0,21.459382
8,8.0,15.168463
9,9.0,155.512823


In [100]:
fig = get_fig(sa_df, x_column="iteration", xlabel="Iterations")
fig.show()

In [85]:
columns = ['time/total_timesteps', 'rollout/ep_rew_mean', 'time/time_elapsed', 'time/episodes']
rl_dfs = []
for i in range(10):
    df = pd.read_csv(f'./data/rl/_{i}/progress.csv')
    df = df[columns].rename(columns={
        'time/total_timesteps': 'timesteps',
        'time/episodes': 'episodes',
        'rollout/ep_rew_mean': 'best_fitness',
        'time/time_elapsed': 'time_elapsed'
    })
    df['run'] = i
    rl_dfs.append(df)
rl_df = pd.concat(rl_dfs)

In [102]:
np.mean([248.6, 321.6, 180.7, 185.4, 98.0, 202.0, 187.9, 481.2, 115.8, 283.8])

230.5

In [86]:
(rl_df.groupby(by='run').agg(
    total_time = ('time_elapsed', 'max')
)).reset_index()

,run,total_time
0,0,128
1,1,129
2,2,138
3,3,135
4,4,159
5,5,201
6,6,163
7,7,151
8,8,167
9,9,159


In [101]:
fig = get_fig(rl_df, x_column="episodes", xlabel="Episodes")
fig.show()

# Hyperparameters

In [107]:
names = ['20,20,1,0.1,0.2_0', '20,50,5,0.1,0.2_1', '20,100,10,0.1,0.2_0']
pop_dfs = []
for i in range(3):
    df = pd.read_csv(f'./data/ea/{names[i]}.csv', index_col=0)
    df['name'] = names[i]
    pop_dfs.append(df)
pop_df = pd.concat(pop_dfs)

In [158]:
(pop_dfs[2].groupby('run').agg(
    total_time = ('total_time', 'max')
) / 1_000_000_000)['total_time'].mean()

57.49413638

In [127]:
fig = get_fig(pop_dfs[0], x_column='generation', xlabel="Generations", color="#0000ff", name="20/1")
fig = get_fig(pop_dfs[1], x_column='generation', xlabel="Generations", color="#00ff00", fig=fig, name="50/5")
fig = get_fig(pop_dfs[2], x_column='generation', xlabel="Generations", color="#ff0000", fig=fig, name="100/10")
fig.show()

In [182]:
names = [
    '20,50,5,0.01,0.02_0', 
    '20,50,5,0.01,0.2_0', 
    '20,50,5,0.1,0.02_0',
    '20,50,5,0.1,0.2_0',
    '20,50,5,1.0,0.02_0',
    '20,50,5,1.0,0.2_0',
    '20,50,5,1.0,1.0_0',
    '20,50,5,1.0,2.0_0'
]
mut_dfs = []
for i in range(len(names)):
    df = pd.read_csv(f'./data/ea/{names[i]}.csv', index_col=0)
    df['name'] = names[i]
    mut_dfs.append(df)

In [185]:
(mut_dfs[6].groupby('run').agg(
    total_time = ('total_time', 'max')
) / 1_000_000_000)['total_time'].mean()

7.171624150000001

In [184]:
#fig = get_fig(mut_dfs[0], x_column='generation', xlabel="Generations", color="#ff0000", name="0.01/0.02")
#fig = get_fig(mut_dfs[1], x_column='generation', xlabel="Generations", color="#0000ff", fig=fig, name="0.01/0.2")
#fig = get_fig(mut_dfs[2], x_column='generation', xlabel="Generations", color="#ff9900", fig=fig, name="0.1/0.02")
# fig = get_fig(mut_dfs[3], x_column='generation', xlabel="Generations", color="#0000ff", name="0.1/0.2")
# fig = get_fig(mut_dfs[4], x_column='generation', xlabel="Generations", color="#ffcc00", fig=fig, name="1.0/0.02")
# fig = get_fig(mut_dfs[5], x_column='generation', xlabel="Generations", color="#ff0000", fig=fig, name="1.0/0.2")
fig = get_fig(mut_dfs[5], x_column='generation', xlabel="Generations", color="#0000ff", name="1.0/0.2")
fig = get_fig(mut_dfs[6], x_column='generation', xlabel="Generations", color="#ff0000", fig=fig, name="1.0/1.0")
fig = get_fig(mut_dfs[7], x_column='generation', xlabel="Generations", color="#00ff00", fig=fig, name="1.0/2.0")
fig.show()